In [1]:
import psycopg
import numpy as np
import pandas as pd 

In [2]:
df = pd.read_csv('book_items_sql.csv')

In [3]:
df.dtypes

Unnamed: 0                int64
product_category         object
product_subcategory      object
rank                     object
url                      object
dist_rating              object
productTitle             object
author                   object
price                    object
bookType                 object
numberOfReview           object
rating                  float64
listeningLength          object
narrator                 object
releaseDate              object
publisher                object
programType              object
version                  object
language                 object
asin                     object
pages                    object
ISBN_10                  object
ISBN_13                  object
weight                   object
dimensions               object
fileSize                 object
textToSpeech             object
screenReader             object
enhanced_typesetting     object
word_Wise                object
x_Ray                    object
dtype: o

In [4]:
df.head()

,Unnamed: 0,product_category,product_subcategory,rank,url,dist_rating,productTitle,author,price,bookType,...,ISBN_10,ISBN_13,weight,dimensions,fileSize,textToSpeech,screenReader,enhanced_typesetting,word_Wise,x_Ray
0,1,Arts & Photography,Sculpture,#1,/100-Magnificent-Mandalas-Beautiful-Relaxation...,"['74%', '15%', '6%', '2%', '2%']",100 Magnificent Mandalas: An Adult Coloring Bo...,Special Art,$11.99,Paperback,...,NaN,979-8722320681,470 g,21.59 x 1.24 x 21.59 cm,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Arts & Photography,Sculpture,#2,/Fashion-Colouring-Book-Girls-Ages/dp/B08K41YH...,"['74%', '15%', '8%', '2%', '1%']",Fashion Colouring Book for Girls Ages 8-12: Go...,Little Eye Twinkle,$7.99,Paperback,...,NaN,979-8691026164,240 g,21.59 x 0.56 x 27.94 cm,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Arts & Photography,Sculpture,#3,/Easy-Origami-John-Montroll/dp/0486272982/ref=...,"['67%', '17%', '10%', '4%', '3%']",Easy Origami,John Montroll,$6.64,Paperback,...,9780486272986,978-0486272986,159 g,20.52 x 0.36 x 27.91 cm,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Arts & Photography,Sculpture,#4,/Origami-Extravaganza-Folding-Paper-Book/dp/08...,"['75%', '15%', '7%', '2%', '1%']","Origami Extravaganza! Folding Paper, a Book, a...",Tuttle Publishing,NaN,Paperback,...,0804832420,978-0804832427,652 g,17.78 x 3.05 x 17.78 cm,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Arts & Photography,Sculpture,#5,/Complete-Book-Origami-Step-Instructions/dp/04...,"['69%', '16%', '9%', '4%', '2%']",The Complete Book of Origami: Step-by-Step Ins...,Robert J. Lang,NaN,Paperback,...,9780486258379,978-0486258379,519 g,22.81 x 1.07 x 30.23 cm,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df.drop(columns='Unnamed: 0')





In [6]:
count = 0
for i in range(len(df)):
    if 'star' in df.iloc[i,4]:
        print('yes',count,i)
        print(df.iloc[i,4])
        count += 1 


In [33]:
conn = psycopg.connect(host='localhost',dbname='data_analysis',user='postgres' ,password='JCr3fr3j92', autocommit=True)

In [34]:
cur = conn.cursor()

In [36]:
create_table = 'CREATE TABLE amazon_books_ver1( book_key integer primary key, book_id integer, book_category varchar(300), book_subcategory varchar(100), rank varchar(40),book_partial_url varchar(500), dist_rating varchar(150), book_title varchar(400), author varchar(300),price varchar(100), book_type varchar(100),number_of_review  varchar(100), rating float ,audible_length varchar(100),narrator varchar(100),release_date  varchar(100),publisher varchar(300),program_type varchar(300),book_version varchar(100),book_language varchar(100),asin varchar(100),page_number varchar(100),ISBN_10 varchar(60),ISBN_13 varchar(60),book_weight varchar(30),book_dimensions varchar(100),file_size varchar(100),text_to_speech varchar(100),screenreader varchar(100),enhanced_typesetting varchar(100),word_wise varchar(100),x_ray varchar(100) )'
drop_table = 'DROP TABLE IF EXISTS amazon_books_ver1;'
##  dist_rating_four varchar(20) ,dist_rating_three varchar(20) ,dist_rating_two varchar(20) ,dist_rating_one varchar(20),


In [37]:
cur.execute(drop_table)
cur.execute(create_table)

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=localhost user=postgres database=data_analysis) at 0x18157dac0>

In [38]:
book_id = 0
unique_book_titles  = pd.unique(df['productTitle'])
for i in range(len((df.iloc[:,]))):
    datapoint = list(map(lambda data: data, df.iloc[i,:]))
    datapoint[0] = str(datapoint[0]).replace("'","''")
    datapoint[1] = str(datapoint[1]).replace("'","''")
    datapoint[2] = str(datapoint[2]).replace("'","''")
    datapoint[4] = datapoint[4].replace("'",'')
    datapoint[5] = str(datapoint[5]).replace("'","''")
    datapoint[6] = str(datapoint[6]).replace("'","''")
    datapoint[8] = str(datapoint[8]).replace("'","''")
    datapoint[12] = str(datapoint[12]).replace("'","''")
    datapoint[14] = str(datapoint[14]).replace("'","''")

    ### 
    book_title = list(map(lambda data: data, df.iloc[i,:]))[5]
    book_id = np.where(book_title == unique_book_titles)[0][0] + 1 
    
    try:
        insert_data = f"INSERT INTO amazon_books_ver1  VALUES ('{i}','{book_id}','{datapoint[0]}','{datapoint[1]}','{datapoint[2]}','{datapoint[3]}' , '{datapoint[4]}','{datapoint[5]}','{datapoint[6]}','{datapoint[7]}','{datapoint[8]}','{datapoint[9]}' ,'{datapoint[10]}','{datapoint[11]}','{datapoint[12]}','{datapoint[13]}','{datapoint[14]}' ,'{datapoint[15]}','{datapoint[16]}','{datapoint[17]}','{datapoint[18]}','{datapoint[19]}','{datapoint[20]}' ,'{datapoint[21]}','{datapoint[22]}','{datapoint[23]}','{datapoint[24]}','{datapoint[25]}' ,'{datapoint[26]}','{datapoint[27]}','{datapoint[28]}','{datapoint[29]}');"
        cur.execute(insert_data)
    except psycopg.errors.StringDataRightTruncation:
        None

In [12]:
datapoint = list(map(lambda data: data, df.iloc[0,:]))

In [39]:
cur.execute('select * from amazon_books_ver1')

<psycopg.Cursor [TUPLES_OK] [IDLE] (host=localhost user=postgres database=data_analysis) at 0x18157dac0>

In [40]:
cur.fetchmany(5)

[(0,
  1,
  'Arts & Photography',
  'Sculpture',
  '#1',
  '/100-Magnificent-Mandalas-Beautiful-Relaxation/dp/B08Z2NV48P/ref=zg_bs_934560_sccl_1/000-0000000-0000000?pd_rd_i=B08Z2NV48P&psc=1',
  '[74%, 15%, 6%, 2%, 2%]',
  '100 Magnificent Mandalas: An Adult Coloring Book with more than 100 Beautiful and Relaxing Mandalas for Stress Relief and Relaxation. (Volume 3)',
  'Special Art',
  '$11.99',
  'Paperback',
  '1149',
  4.6,
  'nan',
  'nan',
  'March 16 2021',
  'Independentlypublished',
  'nan',
  'nan',
  'English',
  'nan',
  '214 pages',
  'nan',
  '979-8722320681',
  '470 g',
  '21.59 x 1.24 x 21.59 cm',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'),
 (1,
  2,
  'Arts & Photography',
  'Sculpture',
  '#2',
  '/Fashion-Colouring-Book-Girls-Ages/dp/B08K41YH1V/ref=zg_bs_934560_sccl_2/000-0000000-0000000?pd_rd_i=B08K41YH1V&psc=1',
  '[74%, 15%, 8%, 2%, 1%]',
  'Fashion Colouring Book for Girls Ages 8-12: Gorgeous Beauty Style Fashion Design Colouring Book for Kids, Girls an

In [78]:
cur.close()